

[참고 코드]
https://github.com/kuangliu/pytorch-cifar/blob/master/models/vgg.py


In [1]:
!mkdir results

mkdir: cannot create directory ‘results’: File exists


In [2]:
!ls

data  results  results_highAcc	sample_data


In [3]:
!cd results
!ls

data  results  results_highAcc	sample_data


In [4]:
!cat results

cat: results: Is a directory


In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy # Add Deepcopy for args
import seaborn as sns 
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainset, valset = torch.utils.data.random_split(trainset, [40000, 10000])
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
partition = {'train': trainset, 'val':valset, 'test':testset}

Files already downloaded and verified
Files already downloaded and verified


In [7]:
testset

Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

print('현재 device :', device)

현재 device : cuda


# 1. Model(MLP)

In [9]:
class MLP(nn.Module):
    
    def __init__(self, in_dim, out_dim, hid_dim, n_layer, act, batch_normal, dropout_p, weight_init):
        super(MLP,self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.hid_dim = hid_dim
        self.n_layer = n_layer  # n_layer = hid_dim(1) + ... + hid_dim(n-1) + out_dim(n)
        self.act = act
        self.batch_normal = batch_normal
        self.dropout = dropout_p
        
        #===Create sequence space===#
        self.linears = nn.ModuleList()
        self.batch_normals = nn.ModuleList()
        
        self.fc1 = nn.Linear(self.in_dim, self.hid_dim)
        for idx in range(n_layer-1):
            self.linears.append(nn.Linear(self.hid_dim, self.hid_dim))
            # 레이어 마다 batch_normalization 시행 예정-> [linear - BN - activation] 반복
            if self.batch_normal == True:
                self.batch_normals.append(nn.BatchNorm1d(hid_dim))
        self.fc2 = nn.Linear(self.hid_dim, self.out_dim)
        
        #===Create Activation Function===#
        if self.act == 'sigmoid':
            self.act = nn.Sigmoid()
        elif self.act == 'relu':
            self.act = nn.ReLU()
        elif self.act == 'tanh':
            self.act = nn.Tanh()
        elif self.act == 'leaky_relu':
            self.act = nn.LeakyReLU()
        else:
            raise ValueError("no valid activation function selected(sigmoid, relu, leaky_relu, tanh)")
            
        #===Create Regularization layer===#
        # dropout
        self.dropout = nn.Dropout(self.dropout)
        # weight_initialization
        if weight_init == 'xavier':
            self.xavier_init()
        elif weight_init == 'he':
            self.he_init()
        else:
            raise ValueError("no valid weight_initializer selected(xavier, he)")
            
    def xavier_init(self):
        for linear in self.linears:
            nn.init.xavier_normal_(linear.weight)
            linear.bias.data.fill_(0.01)
    
    def he_init(self):
        for linear in self.linears:
            torch.nn.init.kaiming_normal_(linear.weight)
            linear.bias.data.fill_(0.01)
        
    def forward(self,x):
        out = self.act(self.fc1(x))
        
                                        #===hidden layer===#
        # 레이어 마다 batch_normalization 시행 예정-> [weight_init - linear - BN - activation - dropout] 반복
        # batch_norm, dropout 은 model.train()에서만 ON
        # batch_norm, dropout 은 hidden layer에서만 적용해야 함!
        for idx in range(len(self.linears)):
            out = self.linears[idx](out)
            if self.batch_normals:
                out = self.batch_normals[idx](out)
            out = self.act(out)
            out = self.dropout(out)
                                        #===hidden layer===#
        
        out = self.fc2(out)
        return out
  

In [10]:
# __init__(self, in_dim, out_dim, hid_dim, n_layer, act, batch_normal, dropout_p, weight_init):

# model = MLP(3072,10,100,4,'leaky_relu',batch_normal=True,dropout_p=0.1,weight_init='he')

# 1. Model(CNN)

In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        
        self.conv_layer1 = nn.Sequential(
            nn.Conv2d(3,64,kernel_size=3,padding=1,stride=1),
            nn.Conv2d(64,256,kernel_size=5,padding=2,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc1 = nn.Linear(65536,10)

    def forward(self,x):
        out = self.conv_layer1(x)
        out = out.view(out.size(0),-1)
        out = self.fc1(out)
        return out



In [12]:
# def dimension_check():
#     model = CNN()
#     x = torch.rand(2,3,32,32)
#     out = model(x)
#     print(out.shape)


In [13]:
# dimension_check()

# 1. Model(CNN_VGG)

In [14]:
# M : maxpooling 단계, int : out channel 숫자 
# M 은 모두 5개 ==>> 전체 width = height = 32 니까, 5번의 maxpooling 통해 receptive field = 1 (32/2/2/2/2/2 = 1) 로 만들어, receptive field = 전체 image로 만들겠다.
# 최종적으로 뽑아낸 복잡한 feature들로 receptive field를 전체 이미지 범위로 놓고, 모든 이미지에 이 feature map을 만들어내겠다. 는 것.
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

In [15]:
# out_dim, args.batch_normal, args.weight_init = 'he', args.act

class CNN_VGG(nn.Module):
    def __init__(self, model_code, in_channels, batch_normal, out_dim, weight_init, act, dropout_p):
        super(CNN_VGG,self).__init__()

        #===Create Activation Function===#
        if act == 'sigmoid':
            self.act = nn.Sigmoid()
        elif act == 'relu':
            self.act = nn.ReLU()
        elif act == 'tanh':
            self.act = nn.Tanh()
        elif act == 'leaky_relu':
            self.act = nn.LeakyReLU()
        else:
            raise ValueError("no valid activation function selected(sigmoid, relu, leaky_relu, tanh)")

        #===Convolutional layer sequence===#
        self.layers = self._make_layers(model_code, in_channels, batch_normal)

        #===Fully-connected layer : [ (weight_init - linear - activation - dropout) - output linear ]===#
        self.dropout = nn.Dropout(dropout_p)
        self.classifier = nn.Sequential(nn.Linear(512*1*1, 256),
                                        self.act,
                                        self.dropout,
                                        nn.Linear(256, out_dim))
        # 마지막 output layer에는 activation function 하지 않는다. softmax가 loss_function에 포함되어있기 때문에.

        #===weight_initialization===#
        if weight_init == 'xavier':
            self.xavier_init()
        elif weight_init == 'he':
            self.he_init()
        else:
            raise ValueError("no valid weight_initializer selected(xavier, he)")
        

    # _ : private method, class 안에서만 활용되는 함수. global scope가 아님.
    def _make_layers(self, model_code, in_channels, batch_normal):

        # (conv + relu) + maxpool
        layers = []
        
        for x in cfg[model_code]:
            # + maxpool
            if x == 'M':
                layers.append(nn.MaxPool2d(2))

            # # + conv + BN + relu
            # else: 
            #     # conv
            #     layers.append(nn.Conv2d(in_channels = in_channels, out_channels = x, kernel_size=3, padding=1, stride=1))
            #     # BN
            #     if batch_normal == True:
            #         layers.append(nn.BatchNorm2d(x))
            #     # relu
            #     layers.append(self.act)
            #     in_channels=x

            # + conv + relu + BN
            else: 
                # conv
                layers.append(nn.Conv2d(in_channels = in_channels, out_channels = x, kernel_size=3, padding=1, stride=1))
                # relu
                layers.append(self.act) 
                # BN
                if batch_normal == True:
                    layers.append(nn.BatchNorm2d(x))
                in_channels=x

        # *layers : layers안에 들어있는 object들을 가변길이로 Sequential method에 argument로 넣기 위해 *붙임
        return nn.Sequential(*layers)

    def forward(self,x):
        out = self.layers(x)
        out = out.view(out.size(0),-1)  # flatten
        out = self.classifier(out)
        return out

    def xavier_init(self):
        for linear in self.classifier:
            if linear != self.act and linear != self.dropout:
                nn.init.xavier_normal_(linear.weight)
                linear.bias.data.fill_(0.01)
    
    def he_init(self):
        for linear in self.classifier:
            if linear != self.act and linear != self.dropout:
                torch.nn.init.kaiming_normal_(linear.weight)
                linear.bias.data.fill_(0.01)


In [16]:
# def dimension_check():
#     model = CNN_VGG('VGG11', in_channels = 3) 
#     x = torch.rand(2,3,32,32)
#     out = model(x)
#     print(out.shape)

In [17]:
# dimension_check()

# 2. Train

In [18]:
def train(model, partition, optimizer, criterion, args):
    # input data preparation
    trainloader = torch.utils.data.DataLoader(partition['train'], 
                                              batch_size=args.train_batch_size, 
                                              shuffle=True, num_workers=2)
    model.train()
    
    train_loss = 0.0
    accuracy_batch = 0.0
    total_sample = 0 
    for i, samples in enumerate(trainloader):
        x_data, y_label = samples
        # Convolutional layer input을 위해, MLP와 달리, 4차원 텐서([256,3,32,32])를 그대로 input([batch,channel,width,height])
        # train, validate, test 모두 flatten input 제거
        # x_data = x_data.view(-1,3072) 
        x_data = x_data.to(device)
        y_label = y_label.to(device)
        
        # forward
        output = model(x_data)
        cost = criterion(output, y_label)
        
        # backward
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        train_loss += cost.item()
        _, predicted_label = torch.max(output, dim=1)
        correct = predicted_label == y_label
        accuracy_batch += correct.float().sum().item()
        
        total_sample += y_label.size(0)
    
    
    # batch 당 평균 loss( len(trainloader) == batch 갯수 )
    train_loss_batch = train_loss / len(trainloader) 
    # 모든 sample의 평균 accuracy
    train_acc_batch = (accuracy_batch / total_sample)*100

    # 학습 후의 model을 return해서, 이후 validate 등에 넣어 활용할 예정
    return model, train_loss_batch, train_acc_batch
  

# 2. Validate

In [19]:
def validate(model, partition, criterion, args):
    valloader = torch.utils.data.DataLoader(partition['val'], 
                                            batch_size=args.test_batch_size, 
                                            shuffle=False, num_workers=2)
    model.eval()
    
    val_loss = 0.0
    accuracy_batch = 0.0
    total_sample = 0
    with torch.no_grad():
        for samples in valloader:
            x_data, y_label = samples
            # x_data = x_data.view(-1,3072)
            x_data = x_data.to(device)
            y_label = y_label.to(device)
            
            # forward
            output = model(x_data)
            cost = criterion(output, y_label)
            
            # backward (X)
            
            val_loss += cost.item()
            _, predicted_label = torch.max(output, dim=1)
            correct = predicted_label == y_label
            accuracy_batch += correct.float().sum().item()
            
            total_sample += y_label.size(0)
            
        val_loss_batch = val_loss / len(valloader) 
        val_acc_batch = (accuracy_batch / total_sample)*100
    
    return val_loss_batch, val_acc_batch

# 3. Test

In [20]:
def test(model, partition, args):
    testloader = torch.utils.data.DataLoader(partition['test'], 
                                             batch_size=args.test_batch_size, 
                                             shuffle=False, num_workers=2)
    model.eval()
    
    accuracy_batch = 0.0
    total_sample = 0
    predicted_labels = []
    y_labels = []
    with torch.no_grad():
        for samples in testloader:
            x_data, y_label = samples
            # x_data = x_data.view(-1,3072)
            x_data = x_data.to(device)
            y_label = y_label.to(device)
            
            # forward (X)

            # backward (X)
            
            output = model(x_data)
            _, predicted_label = torch.max(output, dim=1)
            correct = predicted_label == y_label
            accuracy_batch += correct.float().sum().item()
            
            predicted_labels.append(predicted_label)
            y_labels.append(y_label)

            total_sample += y_label.size(0)
            
        test_acc_batch = (accuracy_batch / total_sample)*100
    
    return test_acc_batch, predicted_labels, y_labels

# 4. Experiment function

In [21]:
def experiment(partition,args):
    
    # model = MLP(args.in_dim, args.out_dim, args.hid_dim,
    #             args.n_layer, args.act,
    #             args.batch_normal, args.dropout_p, args.weight_init)
    
    # model = CNN()

    model = CNN_VGG(model_code=args.model_code, in_channels=args.in_channels,
                    batch_normal = args.batch_normal, out_dim = args.out_dim, 
                    weight_init = args.weight_init, act = args.act,
                    dropout_p = args.dropout_p)
    
    model.to(device)
    
    # Loss function
    criterion = nn.CrossEntropyLoss()
    
    # Optimizer
    if args.optim == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr = args.lr, weight_decay = args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr = args.lr, weight_decay = args.l2)
    elif args.optim == 'ADAM':
        optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError("no valid optimizer selected(SGD, RMSprop, ADAM)")
        
    # Create loss, accuracy list for visualization(seaborn)
    # epoch-wise loss, accuracy
    train_losses, val_losses = [], []
    train_accs, val_accs = [], []
    
    # loop (train / val)
    for epoch in range(args.epoch+1):
        ts = time.time()
        
        model, train_loss_batch, train_acc_batch = train(model, partition, optimizer, criterion, args)
        val_loss_batch, val_acc_batch = validate(model, partition, criterion, args)
        
        te = time.time()
        
        train_losses.append(train_loss_batch)
        val_losses.append(val_loss_batch)
        train_accs.append(train_acc_batch)
        val_accs.append(val_acc_batch)
        
        print('Epoch {}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.2f}/{:2.2f}.\
              Took {:2.2f} sec'.format(epoch, train_acc_batch, val_acc_batch, train_loss_batch, val_loss_batch, te-ts))
    
    test_acc_batch, predicted_labels, y_labels = test(model, partition, args)
    
    # # to keep track of the result of each experiment
    result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['train_accs'] = train_accs
    result['val_accs'] = val_accs
    result['train_acc'] = train_acc_batch
    result['val_acc'] = val_acc_batch
    result['test_acc'] = test_acc_batch
    
    # vars(object) : object가 갖는 attribute를 return! (result의 experiment arguments를 return하기 위함
    # vars(object) : object의 attribute를 dictionary 로 return!
    return vars(args), result, test_acc_batch, predicted_labels, y_labels

# 5. Save and Load

In [22]:
import hashlib
import json
from os import listdir
from os.path import isfile, join
import pandas as pd

def save_exp_result(setting, result):
    exp_name = setting['exp_name'] 
    del setting['epoch']
    del setting['test_batch_size']

    hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
    filename = './results/{}-{}.json'.format(exp_name, hash_key)
    # .updata : dictionary의 append와 같음 (result.update(setting) : result dict + setting dict)
    result.update(setting)
    with open(filename, 'w') as f:
        json.dump(result, f)

    
def load_exp_result(exp_name):
    dir_path = './results'
    filenames = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]
    list_result = []
    for filename in filenames:
        if exp_name in filename:
            with open(join(dir_path, filename), 'r') as infile:
                results = json.load(infile)
                list_result.append(results)
    df = pd.DataFrame(list_result) # .drop(columns=[])
    return df

# 6. Experiment

In [23]:
# # ====== Random Seed Initialization ====== #
# seed = 123
# np.random.seed(seed)
# torch.manual_seed(seed)

# parser = argparse.ArgumentParser()
# args = parser.parse_args("")
# args.exp_name = "exp5_dropout_p_l2"

# # ====== Model Capacity ====== #
# # args.in_dim = 3072  # MLP 입력용
# args.out_dim = 10
# # args.hid_dim = 100  # MLP 입력용
# args.act = 'relu'
# args.model_code = 'VGG13'   # CNN_VGG
# args.in_channels = 3    # CNN_VGG

# # ====== Regularization ======= #
# args.dropout_p = 0.2
# args.batch_normal = True
# args.l2 = 1e-5
# args.weight_init = 'he'

# # ====== Optimizer & Training ====== #
# args.optim = 'ADAM' #'RMSprop' #SGD, RMSprop, ADAM...
# args.lr = 0.002
# args.epoch = 17

# args.train_batch_size = 512
# args.test_batch_size = 1024

# # ====== Experiment Variable ====== #
# name_var1 = 'model_code'
# name_var2 = 'dropout_p'
# list_var1 = ['VGG13','VGG19']
# list_var2 = [0.2]


# for var1 in list_var1:
#     for var2 in list_var2:
#         setattr(args, name_var1, var1)
#         setattr(args, name_var2, var2)
#         print(args)
                
#         setting, result = experiment(partition, deepcopy(args))
#         save_exp_result(setting, result)

In [24]:
# model 1 (highAcc, VGG13, epoch=17, l2_coef = 1e-5, dropout_p = 0.2)
# ====== Random Seed Initialization ====== #
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args1 = parser.parse_args("")
args1.exp_name = "exp6_high_acc_vgg13"

# ====== Model Capacity ====== #
# args.in_dim = 3072  # MLP 입력용
args1.out_dim = 10
# args.hid_dim = 100  # MLP 입력용
args1.act = 'relu'
args1.model_code = 'VGG13'   # CNN_VGG
args1.in_channels = 3    # CNN_VGG

# ====== Regularization ======= #
args1.dropout_p = 0.2
args1.batch_normal = True
args1.l2 = 1e-5
args1.weight_init = 'he'

# ====== Optimizer & Training ====== #
args1.optim = 'ADAM' #'RMSprop' #SGD, RMSprop, ADAM...
args1.lr = 0.002
args1.epoch = 15

args1.train_batch_size = 512
args1.test_batch_size = 1024

# ====== Experiment Variable ====== #
name_var1 = 'model_code'
name_var2 = 'dropout_p'
list_var1 = ['VGG13']
list_var2 = [0.2]



# model 2 (highAcc, VGG19, epoch=17, l2_coef = 1e-5, dropout_p = 0.2)
# ====== Random Seed Initialization ====== #
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args2 = parser.parse_args("")
args2.exp_name = "exp7_high_acc_vgg19"

# ====== Model Capacity ====== #
# args.in_dim = 3072  # MLP 입력용
args2.out_dim = 10
# args.hid_dim = 100  # MLP 입력용
args2.act = 'relu'
args2.model_code = 'VGG19'   # CNN_VGG
args2.in_channels = 3    # CNN_VGG

# ====== Regularization ======= #
args2.dropout_p = 0.2
args2.batch_normal = True
args2.l2 = 1e-5
args2.weight_init = 'he'

# ====== Optimizer & Training ====== #
args2.optim = 'ADAM' #'RMSprop' #SGD, RMSprop, ADAM...
args2.lr = 0.002
args2.epoch = 15

args2.train_batch_size = 512
args2.test_batch_size = 1024

# ====== Experiment Variable ====== #
name_var1 = 'model_code'
name_var2 = 'dropout_p'
list_var1 = ['VGG19']
list_var2 = [0.2]


# model 3 (Reg, VGG13, epoch=13, l2_coef = 1e-3, dropout_p = 0.5)
# ====== Random Seed Initialization ====== #
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args3 = parser.parse_args("")
args3.exp_name = "exp8_Reg_vgg13"

# ====== Model Capacity ====== #
# args.in_dim = 3072  # MLP 입력용
args3.out_dim = 10
# args.hid_dim = 100  # MLP 입력용
args3.act = 'relu'
args3.model_code = 'VGG19'   # CNN_VGG
args3.in_channels = 3    # CNN_VGG

# ====== Regularization ======= #
args3.dropout_p = 0.5
args3.batch_normal = True
args3.l2 = 1e-3
args3.weight_init = 'he'

# ====== Optimizer & Training ====== #
args3.optim = 'ADAM' #'RMSprop' #SGD, RMSprop, ADAM...
args3.lr = 0.002
args3.epoch = 13

args3.train_batch_size = 512
args3.test_batch_size = 1024

# ====== Experiment Variable ====== #
name_var1 = 'model_code'
name_var2 = 'dropout_p'
list_var1 = ['VGG13']
list_var2 = [0.5]


# model 4 (Reg, VGG19, epoch=13, l2_coef = 1e-3, dropout_p = 0.5)
# ====== Random Seed Initialization ====== #
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args4 = parser.parse_args("")
args4.exp_name = "exp9_Reg_vgg19"

# ====== Model Capacity ====== #
# args.in_dim = 3072  # MLP 입력용
args4.out_dim = 10
# args.hid_dim = 100  # MLP 입력용
args4.act = 'relu'
args4.model_code = 'VGG19'   # CNN_VGG
args4.in_channels = 3    # CNN_VGG

# ====== Regularization ======= #
args4.dropout_p = 0.5
args4.batch_normal = True
args4.l2 = 1e-3
args4.weight_init = 'he'

# ====== Optimizer & Training ====== #
args4.optim = 'ADAM' #'RMSprop' #SGD, RMSprop, ADAM...
args4.lr = 0.002
args4.epoch = 13

args4.train_batch_size = 512
args4.test_batch_size = 1024

# ====== Experiment Variable ====== #
name_var1 = 'model_code'
name_var2 = 'dropout_p'
list_var1 = ['VGG19']
list_var2 = [0.5]


args_list = [args1, args2, args3, args4]

predicted_y_list = {}
test_y_list = {}

for idx, args in enumerate(args_list):

    setting, result, test_acc_batch, predicted_labels, y_labels = experiment(partition, deepcopy(args))

    predicted_y_list[idx] = predicted_labels
    test_y_list[idx] = y_labels

    save_exp_result(setting, result)

Epoch 0, Acc(train/val): 26.11/36.01, Loss(train/val) 2.16/1.70.              Took 20.31 sec
Epoch 1, Acc(train/val): 43.72/45.75, Loss(train/val) 1.51/1.49.              Took 20.71 sec
Epoch 2, Acc(train/val): 55.60/53.90, Loss(train/val) 1.23/1.26.              Took 21.46 sec
Epoch 3, Acc(train/val): 62.94/64.60, Loss(train/val) 1.03/0.98.              Took 21.71 sec
Epoch 4, Acc(train/val): 71.73/69.87, Loss(train/val) 0.81/0.87.              Took 21.44 sec
Epoch 5, Acc(train/val): 77.50/73.69, Loss(train/val) 0.65/0.76.              Took 21.43 sec
Epoch 6, Acc(train/val): 81.93/77.99, Loss(train/val) 0.53/0.66.              Took 21.46 sec
Epoch 7, Acc(train/val): 84.38/78.52, Loss(train/val) 0.46/0.66.              Took 21.54 sec
Epoch 8, Acc(train/val): 87.67/77.83, Loss(train/val) 0.37/0.71.              Took 21.45 sec
Epoch 9, Acc(train/val): 90.03/80.83, Loss(train/val) 0.30/0.63.              Took 21.41 sec
Epoch 10, Acc(train/val): 91.85/79.16, Loss(train/val) 0.24/0.73.     

In [66]:
# predicted_y_list ==> 4개 key, dict형, 모든 모델의 pred값 가짐

pred_1 = []
for i in range(len(predicted_y_list[0])):
    pred_1.append((predicted_y_list[0][i]).cpu().numpy())

pred_1_list = np.hstack([pred_1[i] for i in range(10)])


pred_2 = []
for i in range(len(predicted_y_list[1])):
    pred_2.append((predicted_y_list[1][i]).cpu().numpy())

pred_2_list = np.hstack([pred_2[i] for i in range(10)])


pred_3 = []
for i in range(len(predicted_y_list[2])):
    pred_3.append((predicted_y_list[2][i]).cpu().numpy())

pred_3_list = np.hstack([pred_3[i] for i in range(10)])


pred_4 = []
for i in range(len(predicted_y_list[3])):
    pred_4.append((predicted_y_list[3][i]).cpu().numpy())

pred_4_list = np.hstack([pred_4[i] for i in range(10)])


# pred_list = 전체 model의 class ensemble 예측 값
pred_list = np.vstack([pred_1_list, pred_2_list, pred_3_list, pred_4_list])
pred_list.shape


pred_list_max = np.max(pred_list, axis=0)
pred_list_max.shape
pred_list_max[:5]

In [77]:
# test_y_list ==> 4개 key, dict형, 모든 모델의 test_y값 가짐

y_1 = []
for i in range(len(test_y_list[0])):
    y_1.append((test_y_list[0][i]).cpu().numpy())

y_1_list = np.hstack([y_1[i] for i in range(10)])


y_2 = []
for i in range(len(test_y_list[1])):
    y_2.append((test_y_list[1][i]).cpu().numpy())

y_2_list = np.hstack([y_2[i] for i in range(10)])


y_3 = []
for i in range(len(test_y_list[2])):
    y_3.append((test_y_list[2][i]).cpu().numpy())

y_3_list = np.hstack([y_3[i] for i in range(10)])


y_4 = []
for i in range(len(test_y_list[3])):
    y_4.append((test_y_list[3][i]).cpu().numpy())

y_4_list = np.hstack([y_4[i] for i in range(10)])


# y_list = 전체 model의 class ensemble 예측 값
y_list = np.vstack([y_1_list, y_2_list, y_3_list, y_4_list])
y_list.shape


y_list_max = np.max(y_list, axis=0)
y_list_max.shape
y_list_max[:5]

array([3, 8, 8, 0, 6])

In [89]:
compare_list = (pred_list_max == y_list_max)

compare_score = list(map(int,compare_list))

accuracy_score = np.sum(compare_score)/len(compare_score)

print(accuracy_score)

0.7612


In [90]:
# predicted_y_list ==> 4개 key, dict형, 모든 모델의 pred값 가짐

pred_1 = []
for i in range(len(predicted_y_list[0])):
    pred_1.append((predicted_y_list[0][i]).cpu().numpy())

pred_1_list = np.hstack([pred_1[i] for i in range(10)])


pred_2 = []
for i in range(len(predicted_y_list[1])):
    pred_2.append((predicted_y_list[1][i]).cpu().numpy())

pred_2_list = np.hstack([pred_2[i] for i in range(10)])


# pred_list = 전체 model의 class ensemble 예측 값
pred_list = np.vstack([pred_1_list, pred_2_list])

pred_list_max = np.max(pred_list, axis=0)
pred_list_max.shape
pred_list_max[:5]

array([3, 8, 8, 0, 6])

In [91]:
# test_y_list ==> 4개 key, dict형, 모든 모델의 test_y값 가짐

y_1 = []
for i in range(len(test_y_list[0])):
    y_1.append((test_y_list[0][i]).cpu().numpy())

y_1_list = np.hstack([y_1[i] for i in range(10)])


y_2 = []
for i in range(len(test_y_list[1])):
    y_2.append((test_y_list[1][i]).cpu().numpy())

y_2_list = np.hstack([y_2[i] for i in range(10)])


# y_list = 전체 model의 class ensemble 예측 값
y_list = np.vstack([y_1_list, y_2_list])

y_list_max = np.max(y_list, axis=0)
y_list_max.shape
y_list_max[:5]

array([3, 8, 8, 0, 6])

In [96]:
compare_list = (pred_list_max == y_list_max)

compare_score = list(map(int,compare_list))

accuracy_score = np.sum(compare_score)/len(compare_score)

print("Ensemble model accuracy : {} %".format(accuracy_score*100))

Ensemble model accuracy : 81.55 %
